In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_df
from asteroid_element import make_sim_asteroids, get_ast_ref_elts, add_asteroid_elts, get_asteroids
from asteroid_integrate import process_sim
from db_config import db_engine
from db_utils import sp2df

In [3]:
# Typing
from typing import Tuple

In [4]:
epoch = 59000
n0 = 0
n1 = 100
elts = get_ast_ref_elts(epoch=epoch, n0=n0, n1=n1)

In [5]:
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,M,BodyID,BodyName
0,2,84960000,Pallas,59000.0,2.773841,0.229972,0.607949,3.019851,5.414053,2.530303,1000002,SB.Pallas
1,3,84960000,Juno,59000.0,2.668285,0.256936,0.226736,2.964468,4.329572,2.189260,1000003,SB.Juno
2,5,84960000,Astraea,59000.0,2.574037,0.190913,0.093679,2.470881,6.259596,0.311477,1000005,SB.Astraea
3,6,84960000,Hebe,59000.0,2.424533,0.203219,0.257255,2.419784,4.184187,3.328107,1000006,SB.Hebe
4,7,84960000,Iris,59000.0,2.387375,0.230145,0.096370,4.530245,2.534245,4.318395,1000007,SB.Iris
...,...,...,...,...,...,...,...,...,...,...,...,...
92,95,84960000,Arethusa,59000.0,3.064051,0.152789,0.226817,4.241497,2.684048,2.578014,1000095,SB.Arethusa
93,96,84960000,Aegle,59000.0,3.048360,0.141178,0.278942,5.611058,3.638345,3.115040,1000096,SB.Aegle
94,97,84960000,Klotho,59000.0,2.669021,0.257298,0.205558,2.785849,4.689231,0.723346,1000097,SB.Klotho
95,98,84960000,Ianthe,59000.0,2.686497,0.187233,0.271867,6.176797,2.768649,3.757108,1000098,SB.Ianthe


In [6]:
sim = make_sim_asteroids(epoch=epoch, n0=n0, n1=n1)

In [7]:
sim.body_ids

array([     10,       1,       2,     399,     301,       4,       5,
             6,       7,       8,       9, 1000002, 1000003, 1000005,
       1000006, 1000007, 1000008, 1000009, 1000010, 1000011, 1000012,
       1000013, 1000014, 1000015, 1000016, 1000017, 1000018, 1000019,
       1000020, 1000021, 1000022, 1000023, 1000024, 1000025, 1000026,
       1000027, 1000028, 1000029, 1000030, 1000031, 1000032, 1000033,
       1000034, 1000035, 1000036, 1000037, 1000038, 1000039, 1000040,
       1000041, 1000042, 1000043, 1000044, 1000045, 1000046, 1000047,
       1000048, 1000049, 1000050, 1000051, 1000052, 1000053, 1000054,
       1000055, 1000056, 1000057, 1000058, 1000059, 1000060, 1000061,
       1000062, 1000063, 1000064, 1000065, 1000066, 1000067, 1000068,
       1000069, 1000070, 1000071, 1000072, 1000073, 1000074, 1000075,
       1000076, 1000077, 1000078, 1000079, 1000080, 1000081, 1000082,
       1000083, 1000084, 1000085, 1000086, 1000087, 1000088, 1000089,
       1000090, 1000

In [8]:
# sim.body_namesm

In [9]:
mjd0=58900
mjd1=59100
steps_per_day=1
progbar=True

In [10]:
df = integrate_df(sim_epoch=sim, mjd0=mjd0, mjd1=mjd1, steps_per_day=steps_per_day, 
                    save_elements=True, progbar=progbar)

  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
df

,TimeID,BodyID,MJD,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M
0,84816000,10,58900.0,-0.004221,0.007326,0.000035,-0.000008,-0.000002,2.283442e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84816000,1,58900.0,-0.217196,0.256225,0.039910,-0.027079,-0.017177,1.080142e-03,0.387097,0.205652,0.122239,0.843105,0.509335,0.927129,0.627633
2,84816000,2,58900.0,0.047415,0.725334,0.006907,-0.020251,0.001344,1.186871e-03,0.723328,0.006747,0.059246,1.337354,0.957278,-0.795348,-0.785747
3,84816000,399,58900.0,-0.873491,0.478566,0.000017,-0.008494,-0.015196,1.518513e-06,1.000768,0.017385,0.000077,2.886969,-1.065969,0.823827,0.798538
4,84816000,301,58900.0,-0.872089,0.476317,-0.000083,-0.007999,-0.014910,-4.543185e-05,0.002577,0.046343,0.089001,1.691851,1.293892,-3.997685,2.213882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21703,85104000,1000095,59100.0,-3.111452,-1.585699,-0.471925,0.003358,-0.007615,1.489416e-03,3.063683,0.152904,0.226828,-2.041735,2.683601,-3.322059,2.899338
21704,85104000,1000096,59100.0,3.398728,0.113087,0.631092,-0.000909,0.008363,1.714845e-03,3.048257,0.141230,0.278947,-0.672116,-2.645492,3.368376,-2.844074
21705,85104000,1000097,59100.0,-2.410295,0.799996,0.019632,-0.005939,-0.009104,2.209473e-03,2.668847,0.257323,0.205569,2.785837,-1.594280,1.632117,1.118107
21706,85104000,1000098,59100.0,2.910055,0.711917,0.281902,-0.003599,0.008276,2.189298e-03,2.686593,0.187256,0.271865,-0.106390,2.768252,-2.412491,-2.134862


In [15]:
ast = get_asteroids(key_to_body_id=True)

In [16]:
ast

,AsteroidID,BodyID,AsteroidName,BodyName
BodyID,,,,
1000001,1,1000001,Ceres,SB.Ceres
1000002,2,1000002,Pallas,SB.Pallas
1000003,3,1000003,Juno,SB.Juno
1000004,4,1000004,Vesta,SB.Vesta
1000005,5,1000005,Astraea,SB.Astraea
...,...,...,...,...
2413590,1413590,2413590,6013 P-L,SB.6013 P-L
2413591,1413591,2413591,6331 P-L,SB.6331 P-L
2413592,1413592,2413592,6344 P-L,SB.6344 P-L


In [12]:
# Columns for state vectors and orbital element frames
cols_vec = ['TimeID', 'AsteroidID', 'MJD', 'qx', 'qy', 'qz', 'vx', 'vy', 'vz']
cols_elt = ['TimeID', 'AsteroidID', 'MJD', 'a', 'e', 'inc', 'Omega', 'omega', 'f', 'M']

# Mapping to rename columns to match DB; MariaDB has case insensitive column names :(
elt_col_map = {
    'Omega':'Omega_node', 
    'omega': 'omega_peri',
}


In [13]:
mask = df.BodyID > 1000000;
df = df[mask]

In [17]:
df['AsteroidID'] = ast.loc[df.BodyID, 'AsteroidID'].values

In [18]:
df_vec = df[cols_vec]

In [19]:
df_elt = df[cols_elt]
df_elt.rename(columns=elt_col_map, inplace=True)

In [20]:
process_sim(sim=sim, n0=0, n1=100, mjd0=mjd0, mjd1=mjd1, steps_per_day=steps_per_day, truncate=False)


********************************************************************************
Integrating asteroids 000000-000100 from 58900 to 59100...


  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


********************************************************************************
Saving from DataFrame to AsteroidVectors...
Extracting 19497 records from DataFrame into CSV files in chunks of 524288 rows...
CSV file name: ../data/df2db/AsteroidVectors/pid_292675/AsteroidVectors.csv
Elapsed Time for CSV Conversion: 0:00:00.291054
Elapsed Time for DB insertion: 0:00:00.878609

Saving from DataFrame to AsteroidElements...
Extracting 19497 records from DataFrame into CSV files in chunks of 524288 rows...
CSV file name: ../data/df2db/AsteroidElements/pid_292675/AsteroidElements.csv
Elapsed Time for CSV Conversion: 0:00:00.285199
Elapsed Time for DB insertion: 0:00:01.041358


In [ ]:
def csv2db(schema: str, table: str, columns: List[str], i: int, conn):
    """
    Load one CSV file into a staging table for the named DB table.
    INPUTS:
        schema:    Schema of the DB table
        table:     Name of the DB table
        i:         Chunk number
        conn:      DB connection object
    OUTPUTS:
        None.  Modifies the database by inserting from the staging table into the main table
    """

    # Destination and staging table names
    dest_table, staging_table = make_table_names(schema=schema, table=table, i=i)
    # List of column names
    col_list = '(' + ','.join(columns) + ')'
    # Columns to select
    select_fields = ',\n'.join(columns)

    # SQL to insert into the main table from the staging table
    sql_insert_stage = \
        f"""
        REPLACE INTO {dest_table}
        {col_list}
        SELECT
        {select_fields}
        FROM {staging_table};
        """

    # SQL to drop the staging table, which is no longer needed
    sql_drop_stage = f"DROP TABLE {staging_table};"

    # Insert from the staging table, then drop it
    conn.execute(sql_insert_stage)
    conn.execute(sql_drop_stage)

In [22]:
xx = []
for i in range(10):
    x = [i,]
    xx.append(x)



In [23]:
xx

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]

In [24]:
import itertools

In [27]:
list(itertools.chain.from_iterable(xx))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]